In [4]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/SSD/Code/TinyMPC/julia/acrobot/Project.toml`


In [5]:
using Symbolics
using LinearAlgebra
import ForwardDiff as FD

LoadError: UndefVarError: c2 not defined

In [17]:
# ====================
# Explicit model params (working)
# ====================
@variables x[1:4] u
x = collect(x);
x0 = [1; 2; 3; 4]; u0 = 7
# Second-order Bicycle Dynamics
function dynamics(x, u)
    g = 9.81
    m1,m2 = [1; 1]
    l1,l2 = [1; 1]
    J1,J2 = [1; 1]
    θ1,    θ2    = x[1], x[2]
    θ1dot, θ2dot = x[3], x[4]
    s1,c1 = sincos(θ1)
    s2,c2 = sincos(θ2)
    c12 = cos(θ1 + θ2)

    # mass matrix
    m11 = m1*l1^2 + J1 + m2*(l1^2 + l2^2 + 2*l1*l2*c2) + J2
    m12 = m2*(l2^2 + l1*l2*c2 + J2)
    m22 = l2^2*m2 + J2
    M = [m11 m12; m12 m22]

    # bias term
    tmp = l1*l2*m2*s2
    b1 = -(2 * θ1dot * θ2dot + θ2dot^2)*tmp
    b2 = tmp * θ1dot^2
    B = [b1, b2]

    # friction
    c = 1.0
    C = [c*θ1dot, c*θ2dot]

    # gravity term
    g1 = ((m1 + m2)*l2*c1 + m2*l2*c12) * g
    g2 = m2*l2*c12*g
    G = [g1, g2]

    # equations of motion
    τ = [0, u]
    θddot = M\(τ - B - G - C)
    return [θ1dot, θ2dot, θddot[1], θddot[2]]
end
function rk4(x, u)
    dt = 0.05
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    k3 = dt*dynamics(x + k2/2, u)
    k4 = dt*dynamics(x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
function midpoint(x, u)
    dt = 0.05
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    x += k2
end

sym_jac_A = Symbolics.jacobian(rk4(x, u), x);
sym_jac_B = Symbolics.jacobian(rk4(x, u), u);
# # Sub value to validate with ForwardDiff
# sub_dict = Dict()
# [sub_dict[x] = y for (x,y) in zip([x; u], [x0; u0])]
# val_jac_A = substitute.(sym_jac_A, (sub_dict,))
# val_jac_B = substitute.(sym_jac_B, (sub_dict,))
# fd_jac_A = FD.jacobian(_x->rk4(_x, u0), x0)
# fd_jac_B = FD.jacobian(_u->rk4(x0, _u), u0)
# display(norm(fd_jac_A - val_jac_A) + norm(fd_jac_B - val_jac_B))
# display(val_jac_A)
# display(val_jac_B)

LoadError: MethodError: no method matching vec(::Num)

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.

[0mClosest candidates are:
[0m  vec([91m::Distributions.Distribution{var"#s519", S} where {var"#s519"<:Distributions.ArrayLikeVariate, S<:Distributions.ValueSupport}[39m) at /home/khai/.julia/packages/Distributions/WdWXr/src/reshaped.jl:143
[0m  vec([91m::StaticArraysCore.SizedArray{S, T, N, M, TData} where {T, N, M, TData<:AbstractArray{T, M}}[39m) where S at /home/khai/.julia/packages/StaticArrays/pTgFe/src/SizedArray.jl:168
[0m  vec([91m::Transpose{T, var"#s814"} where {T, var"#s814"<:(AbstractVector{T} where T)}[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/adjtrans.jl:239
[0m  ...